## Transfer Learning INCEPTIONV3 using Keras

Please download the dataset from the below url

In [73]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 
    

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [74]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [75]:
tf.__version__

'2.3.0'

In [76]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob


In [77]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = r'C:\Users\50021726\Desktop\Ineuron\catdogdata\training_set'
valid_path = r'C:\Users\50021726\Desktop\Ineuron\catdogdata\test_set'


In [80]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

InceptionV3 = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [81]:
# don't train existing weights
for layer in InceptionV3.layers:
    layer.trainable = False

In [82]:
  # useful for getting number of output classes
folders = glob(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\training_set/*')
folders

['C:\\Users\\50021726\\Desktop\\Ineuron\\catdogdata\\training_set\\cats',
 'C:\\Users\\50021726\\Desktop\\Ineuron\\catdogdata\\training_set\\dogs']

In [83]:
# our layers - you can add more if you want
x = Flatten()(InceptionV3.output)
x

<tf.Tensor 'flatten_3/Reshape:0' shape=(None, 51200) dtype=float32>

In [85]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=InceptionV3.input, outputs=prediction)

In [86]:

# view the structure of the model
model.summary()


Model: "functional_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [87]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [88]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [89]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\training_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 8005 images belonging to 2 classes.


In [90]:
test_set = test_datagen.flow_from_directory(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\test_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                          class_mode = 'categorical')

Found 2023 images belonging to 2 classes.


In [91]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [109]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20
251/251 [==============================] - 71s 285ms/step - loss: 0.2264 - accuracy: 0.9923 - val_loss: 0.6174 - val_accuracy: 0.9862
Epoch 2/20
251/251 [==============================] - 76s 302ms/step - loss: 0.2587 - accuracy: 0.9919 - val_loss: 0.6350 - val_accuracy: 0.9891
Epoch 3/20
251/251 [==============================] - 80s 320ms/step - loss: 0.1344 - accuracy: 0.9940 - val_loss: 1.1958 - val_accuracy: 0.9783
Epoch 4/20
251/251 [==============================] - 79s 316ms/step - loss: 0.3010 - accuracy: 0.9919 - val_loss: 0.6757 - val_accuracy: 0.9896
Epoch 5/20
251/251 [==============================] - 81s 324ms/step - loss: 0.3194 - accuracy: 0.9913 - val_loss: 0.8729 - val_accuracy: 0.9871
Epoch 6/20
251/251 [==============================] - 82s 325ms/step - loss: 0.1478 - accuracy: 0.9940 - val_loss: 0.7248 - val_accuracy: 0.9886
Epoch 7/20
251/251 [==============================] - 82s 327ms/step - loss: 0.1550 - accuracy: 0.9958 - val_loss: 0.8139 - val_ac

In [18]:
# plot the loss
%matplotlib
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

Using matplotlib backend: TkAgg


In [110]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('InceptionV3.h5')

In [111]:

y_pred = model.predict(test_set)


In [112]:
y_pred

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [113]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [114]:
y_pred

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [115]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [116]:
model=load_model('InceptionV3.h5')

In [117]:
img=image.load_img(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\test_set\dogs\dog.4011.jpg',target_size=(224,224))



In [118]:
x=image.img_to_array(img)
x

array([[[ 54.,  69.,  28.],
        [108., 126.,  86.],
        [ 90., 111.,  72.],
        ...,
        [ 25.,  50.,  29.],
        [ 29.,  56.,  41.],
        [ 31.,  57.,  44.]],

       [[ 44.,  61.,  19.],
        [ 45.,  61.,  22.],
        [ 39.,  55.,  16.],
        ...,
        [ 21.,  48.,  31.],
        [ 24.,  55.,  37.],
        [ 21.,  52.,  34.]],

       [[ 50.,  68.,  26.],
        [ 53.,  68.,  29.],
        [ 52.,  62.,  25.],
        ...,
        [ 26.,  53.,  38.],
        [ 23.,  55.,  34.],
        [ 43.,  76.,  55.]],

       ...,

       [[245., 246., 241.],
        [246., 247., 242.],
        [247., 248., 243.],
        ...,
        [194., 193., 199.],
        [191., 191., 193.],
        [199., 199., 199.]],

       [[247., 248., 243.],
        [248., 249., 244.],
        [248., 249., 244.],
        ...,
        [194., 193., 199.],
        [190., 190., 192.],
        [197., 197., 197.]],

       [[250., 251., 246.],
        [250., 251., 246.],
        [249., 2

In [119]:
x.shape

(224, 224, 3)

In [120]:
x=x/255

In [121]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [122]:
model.predict(img_data)

array([[0., 1.]], dtype=float32)

In [123]:
a=np.argmax(model.predict(img_data), axis=1)

In [124]:
a

array([1], dtype=int64)

In [125]:
if(a==0):
    print("The image classified is dog")
else:
    print("The image classified is cat")

The image classified is cat
